# Explicación local

**Descripción**\
Se va a dar explicación a los movimientos incorrectos de un paciente

**Entrada**
* ``raw.csv``:  contiene información detallada sobre cada grabación.


**1. Importar librerías**

In [1]:
import pandas as pd
import functions as fn
import joblib
# Suprimir warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

**2. Input data**

In [2]:
# Dataframe de los datos en bruto
names = ['SubjectID','GestureLabel','GestureName','RepetitionNumber','CorrectLabel','JointName','3D_X','3D_Y','3D_Z']
df_raw = pd.read_csv('../Resultados/raw_pacientes.csv', dtype=object, names=names, skiprows=1136726, nrows=21225)
# coge un solo pacinte realizando un solo gesto

In [3]:
gesture = 'EFL' # vendrá dado por la fase1

**3. Caracterización de movimientos**

In [4]:
# Cálculo de ángulos según gesto
df_calculations = fn.apply_angles_II(df_raw, gesture)
#  Cálculos estadísticos para agrupar por repeticion 
df_stats = fn.calculos_estadisticos(df_calculations)

columnas = ['standardDeviation', 'Maximum', 'Minimum', 'Mean', 'Range',
                'Variance', 'CoV', 'Skewness', 'Kurtosis']
nuevas_columnas = pd.concat([fn.formatear_columnas(df_stats[col], col)
                                for col in columnas], axis=1)
df_stats = pd.concat([df_stats, nuevas_columnas], axis=1)
df_stats = df_stats.drop(columnas, axis=1)

# celimnar correctlabel
df_stats = df_stats.drop(['CorrectLabel'], axis=1)  

**4. Fase 2: Clasificación de la ejecución del movimiento**

In [5]:
correct_mapping = {
            1: 'correcta',
            2: 'incorrecta'
        }

In [6]:
modelo_gesto_path = f'../Resultados/modelo_{gesture}.sav'
best_pipeline, expected_columns = joblib.load(modelo_gesto_path)
df_stats = df_stats.reindex(columns=expected_columns)
correct_labels = best_pipeline.predict(df_stats)

for i,correct_label in enumerate(correct_labels):
    correct_name = correct_mapping.get(correct_label, 'Error en la predición')
    repetition_number = df_stats['RepetitionNumber'].iloc[i]
    if correct_label==1:
        print(f'Repetición {repetition_number}: {correct_name}')
    elif correct_label==2:
        print(f'Repetición {repetition_number}: {correct_name}')
    else:
        print("Error en la predición")

Repetición 1: correcta
Repetición 2: correcta
Repetición 3: correcta
Repetición 4: correcta
Repetición 5: incorrecta
Repetición 6: incorrecta
Repetición 7: correcta
Repetición 8: correcta
Repetición 9: incorrecta
Repetición 10: incorrecta
Repetición 11: incorrecta
Repetición 12: incorrecta
Repetición 13: incorrecta
